In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU, UpSampling2D, add, Input, Conv2DTranspose, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import regularizers
from IPython import display
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

In [3]:
path_1 = "/content/drive/My Drive/resize"
path_2 = "/content/drive/My Drive/down"
valid_images = [".jpg",".jpeg",".png",".tga"]

In [4]:
#
input_img = Input(shape=(64, 64, 3))

l1 = Conv2D(81, (5, 5), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(input_img)
l1 = LeakyReLU()(l1)
l2 = Conv2D(54, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l1)
l2 = LeakyReLU()(l2)
l3 = Conv2D(54, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l2)
l3 = LeakyReLU()(l3)
l4 = Conv2D(54, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l3)
l4 = LeakyReLU()(l4)
l5 = Conv2DTranspose(filters=54, kernel_size=(3, 3), strides=(2, 2), padding='same')(l4)
l5 = BatchNormalization()(l5)

l6 = Conv2D(27, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l5)
l6 = LeakyReLU()(l6)
l7 = Conv2D(9, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l6)
l7 = LeakyReLU()(l7)

l8 = Conv2DTranspose(filters=3, kernel_size=(3, 3), strides=(2, 2), padding='same')(l7)
#channel = 3, for RGB
l9 = Conv2D(3, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l8)
last = LeakyReLU()(l9)
SRCNN = Model(input_img, last)
SRCNN.compile(optimizer='adadelta', loss='mse')
SRCNN.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 81)        6156      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 81)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 54)        39420     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 54)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 54)        26298     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 54)       

In [5]:
x_train_n = np.load('/content/drive/My Drive/x_n.npy')
x_train_down = np.load('/content/drive/My Drive/resize_64.npy')
x_train_down.shape

(5000, 64, 64, 3)

In [ ]:
SRCNN.fit(x_train_down, x_train_n, epochs=100, batch_size=10, shuffle=True, validation_split=0.20)

Epoch 1/100
400/400 [==============================] - 16s 39ms/step - loss: 18323.0488 - val_loss: 18143.5000
Epoch 2/100
400/400 [==============================] - 16s 39ms/step - loss: 18092.9121 - val_loss: 17716.5645
Epoch 3/100
400/400 [==============================] - 16s 40ms/step - loss: 14258.9160 - val_loss: 8180.4165
Epoch 4/100
400/400 [==============================] - 16s 41ms/step - loss: 2300.2556 - val_loss: 1470.7609
Epoch 5/100
400/400 [==============================] - 16s 41ms/step - loss: 1213.4038 - val_loss: 1011.7049
Epoch 6/100
400/400 [==============================] - 16s 41ms/step - loss: 1041.4362 - val_loss: 919.6134
Epoch 7/100
400/400 [==============================] - 16s 40ms/step - loss: 978.9932 - val_loss: 872.2773
Epoch 8/100
400/400 [==============================] - 16s 40ms/step - loss: 938.8779 - val_loss: 856.7849
Epoch 9/100
400/400 [==============================] - 16s 40ms/step - loss: 921.9553 - val_loss: 838.8282
Epoch 10/100
400/400 

In [19]:
x_d = x_train_down[0:2]
srl = SRCNN.predict(x_d)
print(srl.shape)

(2, 256, 256, 3)


In [20]:
loss_and_metrics = SRCNN.evaluate(x_d, x_train_n[0:2])
print('')
print('loss_and_metrics : ' + str(loss_and_metrics)) 

1/1 [==============================] - 0s 1ms/step - loss: 425.2245

loss_and_metrics : 425.2244873046875


In [21]:
#translate color
def translate_color(img):
  b, g, r = cv2.split(img)   # img파일을 b,g,r로 분리
  img2 = cv2.merge([r,g,b]) # b, r을 바꿔서 Merge
  return img2

In [24]:
image_index = 0
i = 1
plt.figure(figsize=(128, 128))
ax = plt.subplot(10,10,i)
plt.imshow(translate_color(x_train_down[image_index]))
i+=1
ax = plt.subplot(10,10,i)
plt.imshow(translate_color(x_train_down[image_index]), interpolation="bicubic")
i+=1
ax = plt.subplot(10,10,i)
plt.imshow(srl[image_index].astype(np.uint16))
i+=1
ax = plt.subplot(10,10,i)
plt.imshow(translate_color(srl[image_index].astype(np.uint64)))
i+=1
ax = plt.subplot(10,10,i)
plt.imshow(translate_color(x_train_n[image_index]))
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [26]:
cv2.imwrite("model_1_epochs200.jpg", srl[0])

True

In [ ]:
#
input_img = Input(shape=(64, 64, 3))

l1 = Conv2D(64, (5, 5), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(input_img)
l1 = LeakyReLU()(l1)
l2 = Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l1)
l2 = LeakyReLU()(l2)
l3 = Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l2)
l3 = LeakyReLU()(l3)
l4 = Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l3)
l4 = LeakyReLU()(l4)
l5 = Conv2DTranspose(filters=8, kernel_size=(3, 3), strides=(2, 2), padding='same')(l4)
l5 = BatchNormalization()(l5)

l6 = Conv2D(4, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l5)
l6 = LeakyReLU()(l6)

l7 = Conv2DTranspose(filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same')(l6)
#channel = 3, for RGB
l8 = Conv2D(3, (3, 3), padding='same', kernel_initializer='glorot_normal', activity_regularizer=regularizers.l1(10e-10))(l7)
last = LeakyReLU()(l8)
SRCNN = Model(input_img, last)
SRCNN.compile(optimizer='adadelta', loss='mse')
SRCNN.summary()